In [7]:
# from libs.alibaba.visitor import Visitor
from libs.alibaba.alibaba import Alibaba

from libs.json import JSON
from openpyxl import load_workbook

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
from libs.CeleryTasks import tasks
import html
import time
import re
import os
import traceback
import types
import pendulum

In [3]:
class BusinessCard:
    api = 'https://profile.alibaba.com/receive_list.htm'
    browser = None
    
    def __init__(self, market, account=None, headless=True, browser=None):
        self.market = market
        self.account = account
        self.lid = account['lid'] if account else market['lid']
        self.lpwd = account['lpwd'] if account else market['lpwd']
        self.lname = account['lname'] if account else market['lname']
        self.mobile = account['mobile'] if account else market['mobile']

        self.account = {}
        self.account['lid'] = self.lid
        self.account['lpwd'] = self.lpwd
        self.account['lname'] = self.lname
        self.account['mobile'] = self.mobile

        self.confi_dir = market['directory'] + '_config'

        self.alibaba = None
        self.browser = browser
        self.headless = headless

        self.mail_message = "Hi,\\nNice Day. This is Ada.\\nThanks for your visit to our products.\\nWould you pls tell us your WhatsApp number? I would like to send our product catalog and price list to you. Thanks\\nMy WhatsApp  is +8618563918130.\\n\\nRegards\\nAda"

    def load_url(self):
        while True:
            try:
                if self.alibaba is None:
                    alibaba = Alibaba(self.lid, self.lpwd, headless=self.headless, browser=self.browser)
                    alibaba.login()
                    if self.browser is None:
                        self.browser = alibaba.browser
                    self.alibaba = alibaba

                self.browser.get(self.api)
                if 'login.alibaba.com' in self.browser.current_url:
                    self.alibaba.login()
                    self.browser.get(self.api)

                WebDriverWait(self.browser, 15).until(
                    EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.bp-loading-panel')))

                # try to close all follow-me-popups
                while True:
                    btn_close = self.browser.find_elements_by_css_selector('div.follow-me-close')
                    if btn_close:
                        webdriver.ActionChains(self.browser).send_keys(Keys.ESCAPE).perform()
                        # wait 1 second to see if new popup commes
                        self.browser.implicitly_wait(1)
                        continue
                    else:
                        break
                break
            except WebDriverException as e:
                if 'chrome not reachable' in str(e):
                    print(str(e))
                    self.browser = None
                continue

In [8]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
bc = BusinessCard(market, headless=False)

. storage markets.json
.\storage\markets.json was deserialized!


In [11]:
bc.load_url()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [49]:
def update(self):
    self.load_url()
    
    while True:
        div_card_list = self.browser.find_element_by_css_selector('div#J-card-list')
        btn_next = div_card_list.find_element_by_css_selector('div.J-list-pagination a.next, div.J-list-pagination span.next')
        
        lis = div_card_list.find_elements_by_css_selector('div.list-body li.list-item')
        for li in lis:
            card = {}
            _li = pq(li.get_attribute('outerHTML'))
            
            card['img'] = _li.find('div.avatar a.imgbox img').attr('src')
            card['href'] = _li.find('div.avatar a.imgbox').attr('href')
            
            card['name'] = _li.find('div.desc a.name').text()
            card['title'] = _li.find('div.desc div.title').text()
            card['email'] = _li.find('div.desc div.email').attr('title')
            card['company'] = _li.find('div.desc a.company').attr('title')
            card['company_href'] = _li.find('div.desc a.company').attr('href')
            card['identity'] = _li.find('div.bi-icon a.scc-business-identity').attr('data-status')
            card['country_code'] = _li.find('div.bi-icon span.country-name').text()
            card['country_name'] = _li.find('div.bi-icon span.country-name').parent().attr('data-country')
            card['country_icon'] = _li.find('div.bi-icon i.country-ico img').attr('src')
            card_type = _li.find('div.bi-icon span.type')
            if card_type:
                card['type'] = card_type.attr('title')
            else:
                card['type'] = None
            card['time'] = _li.find('div.time').text()
            for line in _li.find('div.card-back div.line'):
                line = pq(line)
                key = line.find('span.key').text()
                val = line.find('span.val').text()
                if 'email' in key.lower():
                    continue
                key = key.split(':')[0]
                card[key] = val

            btn_send_card = li.find_elements_by_css_selector('a.J-send-card')
            if btn_send_card:
                btn_send_card[0].click()
                
            print(card)
        break
        if 'disable' in btn_next.get_attribute('class'):
            break
        else:
            btn_next.click()
            div_loading_panel = WebDriverWait(self.browser, 15).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'div.ui2-loading-panel')))
            WebDriverWait(self.browser, 5).until(EC.staleness_of(div_loading_panel))
bc.update = types.MethodType(update, bc)

In [50]:
bc.update()

{'img': None, 'href': '//profile.alibaba.com/index.htm?m=B1PID1UIUecUxAsxkFF2a_NtzYz-72yqNmPRbxgPP2d-t_uXY*', 'name': 'Dmitry Ivanov', 'title': '', 'email': 'lovely-boss@mail.ru', 'company': 'No Company Name', 'company_href': 'https://ru1240163655unkq.fm.alibaba.com/company_profile.html', 'identity': 'c', 'country_code': 'RU', 'country_name': 'Russian Federation', 'country_icon': '//is.alicdn.com/simg/single/nflag/ru.gif', 'type': 'Other', 'time': 'Dec 4, 2018 \xa0From Inquiry', 'Tel': '7-929-3690466', 'Fax': '', 'Mobile': '', 'Website': ''}
{'img': None, 'href': '//profile.alibaba.com/index.htm?m=B1PID1IWqvqrDmQy5g24pt4P-sr5QcspmtCIFCeT78bj1_Vag*', 'name': 'Tracy Fraley', 'title': '', 'email': 'tracyfraley@hotmail.com', 'company': 'No Company Name', 'company_href': 'https://us1094064740.fm.alibaba.com/company_profile.html', 'identity': 'a', 'country_code': 'US', 'country_name': 'United States', 'country_icon': '//is.alicdn.com/simg/single/nflag/us.gif', 'type': 'Other', 'time': 'Dec 4